# Fluent Python
https://github.com/fluentpython/example-code

파이썬 용어집 

https://docs.python.org/ko/3/glossary.html

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

def time_check(func):
    def decorated():
        import time
        start = time.time()
        func()
        print("---{}s seconds---".format(time.time()-start_time))
    return decorated# Fluent Python

## CHAPTER 7 Function Decorators and Closures

### decorator 101
모듈이 로딩될때, 임포트 타입에 실행 

함수 decorator 는 모듈이 임포트 되자마자 실행되지만, decorated function 은 명시적으로 호출될떄만 실해ㅇ된다. 

import time, runtime 의 차이 

```python
import registration
running register(<function f1 at 0x10063b1e0>)
running register(<function f2 at 0x10063b268>)
```

In [2]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [3]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

함수 본체를 컴파일 할때  가 함수 안에서 할당되므로 b 를 지역 변수로 판다.

a design choice: Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local.

In [5]:
from dis import dis
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


### closure

a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there. It does not matter whether the function is anonymous or not; **what matters is that it can access nonglobal variables that are defined outside of its body.**

* free variable: This is a technical term meaning a variable that is not bound in the local scope

외부함수가 내부함수를 리턴하고 종료하더라도 리턴된 내부함수는 여전히 외부함수의 변수를 사용 가능하다.


In [6]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [7]:
avg = make_averager()
avg.__code__.co_varnames
avg.__code__.co_freevars

('new_value', 'total')

('series',)

In [9]:
avg.__closure__
avg(10)
avg.__closure__[0].cell_contents

(<cell at 0x7f3eabbcfaf8: list object at 0x7f3eab8cbcc8>,)

10.0

[10]

### The nonlocal Declaration

nonlocal 변수 = free var

free variable : 어떤 코드블록 안에서 사용되지만, 글로벌 변수도 아니고 그 블록 내에 정의하지도 않은 변수[

In [10]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1 # 내부에서 할당이 가능해짐.
        total += new_value
        return total / count
    return averager

In [ ]:
import time 

def clock(func):
    def clocked(*args): #
        t0 = time.perf_counter()
        result = func(*args) #
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [18]:
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [22]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n, a="a", b="b"):
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6, a="b", b=1))

**************************************** Calling snooze(.123)
[0.12327361s] snooze(0.123) -> None 
**************************************** Calling factorial(6)
[0.00000191s] factorial(1) -> 1 
[0.00008965s] factorial(2) -> 2 
[0.00015521s] factorial(3) -> 6 
[0.00020814s] factorial(4) -> 24 
[0.00027275s] factorial(5) -> 120 
[0.00033760s] factorial(6, a='b', b=1) -> 720 
6! = 720


### Decorators in the Standard Library

제공하는 내장함수
- property()
- classmethod()
- staticmethod()
- functools.wraps()


### Memoization with functools.lru_cache()

LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

In [23]:
@functools.lru_cache() # parameters 때문에 일반 함수처럼 호출. 
@clock # 누적된 데코레이터
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0 
[0.00000119s] fibonacci(1) -> 1 
[0.00706863s] fibonacci(2) -> 1 
[0.00000429s] fibonacci(3) -> 2 
[0.00776672s] fibonacci(4) -> 3 
[0.00000286s] fibonacci(5) -> 5 
[0.00840569s] fibonacci(6) -> 8 
8


`functools.lru_cache(maxsize=128, typed=False)`

The maxsize argument determines how many call results are stored. After the cache is full, older results are discarded to make room. For optimal performance, maxsize should
be a power of 2.

### Generic Functions with Single Dispatch

함수 오버로딩이 가능하도록 하는 기능 

In [25]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral) # 새로운 자료형에 대한 처리
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence) # 여러 자료형에 대한 처리
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

### A Parameterized Registration Decorator



In [34]:
registry = set()

def register(active=True): # register takes an optional keyword argument.
    def decorate(func): # 함수를 인수로 받음, 실제 decorator
        print('running register(active=%s)->decorate(%s)'% (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x7f3eab8d2f28>)
running register(active=True)->decorate(<function f2 at 0x7f3eab89d048>)


In [35]:
registry
register()(f3)
register(active=False)(f2)
registry

{<function __main__.f2()>}

running register(active=True)->decorate(<function f3 at 0x7f3ed8726730>)


<function __main__.f3()>

running register(active=False)->decorate(<function f2 at 0x7f3eab89d048>)


<function __main__.f2()>

{<function __main__.f3()>}

### The Parameterized Clock Decorator


In [36]:
import time
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'
def clock(fmt=DEFAULT_FMT):  # our parameterized decorator factory.
    def decorate(func):      # the actual decorator.
        def clocked(*_args): # wraps the decorated function.
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

\**locals() 를 사용하면 fmt 가 clocked() 의 지역 변수를 모두 참조 할수 있게 해줌.

In [37]:
@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12318420s] snooze(0.123) -> None
[0.12329173s] snooze(0.123) -> None
[0.12319469s] snooze(0.123) -> None


In [38]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12400031089782715s
snooze: 0.12324666976928711s
snooze: 0.12319064140319824s


How you implemented your Python decorator is wrong

https://github.com/GrahamDumpleton/wrapt/blob/develop/blog/01-how-you-implemented-your-python-decorator-is-wrong.md

Decorators (2014)
- 01 - How you implemented your Python decorator is wrong - (7th January 2014)
- 02 - The interaction between decorators and descriptors - (7th January 2014)
- 03 - Implementing a factory for creating decorators - (8th January 2014)
- 04 - Implementing a universal decorator - (9th January 2014)
- 05 - Decorators which accept arguments - (11th January 2014)
- 06 - Maintaining decorator state using a class - (13th January 2014)
- 07 - The missing @synchronized decorator - (14th January 2014)
- 08 - The @synchronized decorator as context manager - (15th January 2014)
- 09 - Performance overhead of using decorators - (8th February 2014)
- 10 - Performance overhead when applying decorators to methods - (17th February 2014)

Monkey Patching (2015)
- 11 - Safely applying monkey patches in Python - (11th March 2015)
- 12 - Using wrapt to support testing of software - (12th March 2015)
- 13 - Ordering issues when monkey patching in Python - (18th March 2015)
- 14 - Automatic patching of Python applications - (9th April 2015)

https://github.com/GrahamDumpleton/wrapt/blob/develop/blog/README.md